In [1]:
import pypesto
import pypesto.petab
import pypesto.optimize as optimize
import pypesto.sample as sample
import pypesto.visualize as visualize

import petab
import numpy as np
from scipy.special import gammaln
from scipy.special import gamma
import pickle
from scipy import integrate
from copy import deepcopy

datatype = "original"

# import to petab
if datatype == "original":
    petab_problem = petab.Problem.from_yaml(
    "corrupted_data/SS_conversion_reaction_original.yaml")
elif datatype == "switch":
    petab_problem = petab.Problem.from_yaml(
    "corrupted_data/SS_conversion_reaction_switch.yaml")
else:
    petab_problem = petab.Problem.from_yaml(
    "corrupted_data/SS_conversion_reaction_loss .yaml")

In [2]:
def analytical_b(t, a0, b0, k1, k2):
    return (k2 - k2 * np.exp(-(k2 + k1) * t)) / (k2 + k1)

def simulate_model(x, tvec):
    # assign parameters
    k1, k2, _ = x
    # define initial conditions
    a0 = 1
    b0 = 0
    # simulate model
    simulation = [analytical_b(t, a0, b0, k1, k2)
                   for t in tvec]
    return simulation

After importing the model, we need to define the objective function. This time we will do it via an external function that will be used then by pyPESTO instead of using the built-in ones.

For numerical reasons we will implement the log likelihood and log prior.

For calculation-reasons we renumber $y_k$ and $h_k$ so that $\hat{y}_k - h_k$ are ordered from smallest to biggest, i.e. $y_1 - h_1$ is the smallest number, $y_N - h_N$ the biggest.
Then we choose $b_0 = -\infty, b_i = y_i - h_i (i = 1, \ldots, N), b_{N+1} = \infty$. Now we can split the integral in the following parts:
\begin{align*}
    \frac{p(\sigma)}{2\sigma} \left( \sum_{i = 0}^N \int_{b_i}^{b_{i+1}}  \exp \left\{- \frac{\sum_{k = 1}^N {|c - b_k|}}{\sigma} \right\} p(c)  dc \right) 
\end{align*}
with $p(\sigma) = 1$.

In [3]:
def log_prior(offset):
    """ Log prior function."""
    # assign variables from input x
#     scale = x[2]
#     offset = x[3]


    # exponential prior
    exp_prior = np.log(lamda) - lamda * offset

    #Gaussian prior
    #Gauss_prior = np.log(offset) - (np.log(2) + np.log(np.pi)) / 2 - np.log(tau) - ((offset - mu)/tau)**2 /2

    #Laplacian prior
    #Lapl_prior = np.log(1) - np.log(2) - np.log(tau) - abs(offset - mu) / tau
    
    #Amoroso prior
    #Amo_prior = - gammaln(c) + np.log(abs(d / b)) + (c * d - 1) * (np.log(scale) - np.log(b)) - (scale / b)**d

    return exp_prior


def negative_log_posterior(offset,x):
    """ Negative log posterior function."""

    scale = x[2]
#     offset = x[3]

    # experimental data
    data = np.asarray(petab_problem.measurement_df.measurement)
    # time vector
    tvec = np.asarray(petab_problem.measurement_df.time)

    N = len(tvec)

    # simulate model
    _simulation = simulate_model(np.exp(x), tvec)
    simulation = np.asarray(_simulation)
    
    # evaluate standard log likelihood
    res = data - simulation
    b_vector = np.sort(res)
    
    sum_res = np.sum(abs(offset-b_vector)) / scale

    l_llh = - sum_res

    # evaluate log normal-gamma prior
    l_prior = log_prior(offset)

    # return log posterior
    return (l_llh + l_prior)

def evaluate_posterior_standard(offset,x):
    # evaluate log posterior
    _P = negative_log_posterior(offset, x)
    
    # transform to posterior (not log)
    return np.exp(_P)

def numerical_marginalisation_offset(x):
    scale = x[2]
    
    tvec = np.asarray(petab_problem.measurement_df.time)
    N = len(tvec)
    
    marginal_posterior, _ = integrate.quad(evaluate_posterior_standard, -np.inf, np.inf,args=(x))

    log_marginal_posterior = np.log(marginal_posterior)+N*(- np.log(2) -np.log(scale))
    
    return -log_marginal_posterior

In [4]:
x0 = np.array([-1.2741, -0.6160, 0.3684])
lamda = 0.01

print('Correct: 4.163947227276439')
print('Obtained: '+str(numerical_marginalisation_offset(x0)))

Correct: 4.163947227276439
Obtained: 4.1639480977344885


\begin{align}
     \frac{p(\sigma)}{2\sigma} \left ( \sum_{i = 0}^N \exp \left\{\frac{\bigl( \sum_{k = 1}^i b_k \bigr) - \bigl( \sum_{k = i + 1}^N b_k \bigr)}{\sigma}\right\} \int_{b_i}^{b_{i + 1}} e^{\frac{c \cdot (N - 2i)}{\sigma}} p(c) dc \right )
\end{align}
with $p(\sigma)=1$

In [5]:
def log_prior(offset):
    """ Log prior function."""

    # exponential prior
    exp_prior = np.log(lamda) - lamda * offset
    return exp_prior


def negative_log_posterior(offset,x,N,i):
    """ Negative log posterior function."""

    scale = x[2]

    l_llh = offset*(N-2*i)/scale

    # evaluate log normal-gamma prior
    l_prior = log_prior(offset)

    # return log posterior
    return (l_llh + l_prior)

def evaluate_posterior_standard(offset,x,N,i):
    # evaluate log posterior
    _P = negative_log_posterior(offset, x,N,i)
    
    # transform to posterior (not log)
    return np.exp(_P)

def numerical_marginalisation_offset(x):
    scale = x[2]
    
    data = np.asarray(petab_problem.measurement_df.measurement)
    tvec = np.asarray(petab_problem.measurement_df.time)
    N = len(tvec)
    
    # simulate model
    _simulation = simulate_model(np.exp(x), tvec)
    simulation = np.asarray(_simulation)
    
    # evaluate standard log likelihood
    res = data - simulation
    b_vector = np.sort(res)
    
    bounds = np.append(np.append(-np.inf, b_vector), np.inf)
    
    marginal_posterior = 0
    for n in range(len(bounds)-1):
        _marginal_posterior, _ = integrate.quad(evaluate_posterior_standard, 
                                                bounds[n], bounds[n+1], 
                                                args=(x,N,n))
        
        aux = np.sum(b_vector[:n]) - np.sum(b_vector[n:])

        marginal_posterior += _marginal_posterior*np.exp(aux/scale)
    
    log_marginal_posterior = np.log(marginal_posterior)
    log_marginal_posterior += N*(- np.log(2) -np.log(scale))
    
    return -log_marginal_posterior

In [6]:
x0 = np.array([-1.2741, -0.6160, 0.3684])
lamda = 0.01

print('Correct: 4.163947227276439')
print('Obtained: '+str(numerical_marginalisation_offset(x0)))

Correct: 4.163947227276439
Obtained: 4.163947167834274


In [7]:
def negative_log_posterior(offset,x,N,i):
    """ Negative log posterior function."""

    scale = x[2]

    l_llh = offset*((N-2*i)/scale - lamda)

    # return log posterior
    return l_llh

def evaluate_posterior_standard(offset,x,N,i):
    # evaluate log posterior
    _P = negative_log_posterior(offset, x,N,i)
    
    # transform to posterior (not log)
    return np.exp(_P)

def numerical_marginalisation_offset(x):
    scale = x[2]
    
    data = np.asarray(petab_problem.measurement_df.measurement)
    tvec = np.asarray(petab_problem.measurement_df.time)
    N = len(tvec)
    
    # simulate model
    _simulation = simulate_model(np.exp(x), tvec)
    simulation = np.asarray(_simulation)
    
    # evaluate standard log likelihood
    res = data - simulation
    b_vector = np.sort(res)
    
    bounds = np.append(np.append(-np.inf, b_vector), np.inf)
    r = np.argmax(bounds >= 0)-1
    
    marginal_posterior = 0
    for n in range(len(bounds)-1):
        if n < r:
            _marginal_posterior, _ = integrate.quad(evaluate_posterior_standard, 
                                                    0, bounds[r], 
                                                    args=(x,N,r))
            aux = np.sum(b_vector[:r]) - np.sum(b_vector[r:])
        else:
            _marginal_posterior, _ = integrate.quad(evaluate_posterior_standard, 
                                                    bounds[n], bounds[n+1], 
                                                    args=(x,N,n))
        
            aux = np.sum(b_vector[:n]) - np.sum(b_vector[n:])

        marginal_posterior += _marginal_posterior*np.exp(aux/scale)
    
    log_marginal_posterior = np.log(marginal_posterior)
    log_marginal_posterior += N*(- np.log(2) -np.log(scale))+np.log(lamda) 
    
    return -log_marginal_posterior

In [8]:
x0 = np.array([-1.2741, -0.6160, 0.3684])
lamda = 0.01

print('Correct: 4.163947227276439')
print('Obtained: '+str(numerical_marginalisation_offset(x0)))

Correct: 4.163947227276439
Obtained: 4.163947167834275


\begin{align*}
& \frac{\lambda \cdot p(\sigma)}{2\sigma}  \Biggl(  e^{l_{r}/\sigma} \frac{\sigma}{N -2r - \sigma\lambda} \cdot \Biggl( \exp\biggl(b_{r + 1} \cdot \left(\frac{N - 2r}{\sigma} - \lambda\right)\biggr) - 1 \Biggr) \label{eq 99} \\
    &+ \sum_{i = r + 1}^{N-1} e^{l_i/\sigma} \frac{\sigma}{N - 2i - \sigma\lambda} \Biggl(\exp\biggl(b_{i + 1} \cdot \left(\frac{N - 2i}{\sigma} - \lambda\right)\biggr) - \exp\biggl(b_i \cdot \left(\frac{N - 2i}{\sigma} - \lambda\right)\biggr) \Biggr) \nonumber \nonumber\\
    &+ e^{l_N/\sigma} \frac{\sigma}{N + \sigma\lambda} \exp\biggl(b_N \cdot \left(\frac{-N}{\sigma} - \lambda\right)\biggr) \Biggr) 
\end{align*}

In [9]:
def numerical_marginalisation_offset(x):
    scale = x[2]
    
    data = np.asarray(petab_problem.measurement_df.measurement)
    tvec = np.asarray(petab_problem.measurement_df.time)
    N = len(tvec)
    
    # simulate model
    _simulation = simulate_model(np.exp(x), tvec)
    simulation = np.asarray(_simulation)
    
    # evaluate standard log likelihood
    res = data - simulation
    b_vector = np.sort(res)
    
    bounds = np.append(np.append(-np.inf, b_vector), np.inf)
    r = np.argmax(bounds >= 0)-1
    
    marginal_posterior = 0
    for n in range(len(bounds)-1):
        l_value = np.sum(b_vector[:n]) - np.sum(b_vector[n:])
        tmp = l_value/scale
        if n < r:
            aux1 = scale/(N-2*r-scale*lamda)
            aux2 = (np.exp(tmp+bounds[r]*((N-2*r)/scale-lamda))-np.exp(tmp))

        elif n == len(bounds)-2:
            aux1 = scale/(N+scale*lamda)
            aux2 = np.exp(tmp+bounds[n]*(-N/scale-lamda))

        else:
            aux = (N-2*n)/scale-lamda
            aux1 = scale/(N-2*n-scale*lamda)
            aux2 = np.exp(tmp+bounds[n+1]*aux)-np.exp(tmp+bounds[n]*aux)
        
        marginal_posterior += aux1*aux2
        
    log_marginal_posterior = np.log(marginal_posterior)
    log_marginal_posterior += N*(- np.log(2) -np.log(scale))+np.log(lamda) 
    
    return -log_marginal_posterior

In [10]:
x = np.array([-1.66794275, -0.5614265, 0.01556866])
lamda = 0.01

print('Obtaind: '+str(numerical_marginalisation_offset(x)))

Obtaind: -18.142933010755645


In [11]:
x0 = np.array([-1.2741, -0.6160, 0.3684])
lamda = 0.01

print('Correct: 4.163947227276439')
print('Obtaind: '+str(numerical_marginalisation_offset(x0)))

Correct: 4.163947227276439
Obtaind: 4.1639471678254525
